In [1]:
import numpy as np 
from random import shuffle
import pandas as pd
from glob import glob

In [2]:
def pick_next_dialogue(data, content):
    res = np.random.choice(data)
    df = pd.read_csv("./data/mid_merge_f/"+res)
    temp_content = set(list(pd.unique(df['A_inst']))+list(pd.unique(df['B_inst'])))
    
    cpt = 0
    while len(content.intersection(temp_content)) != 0: 
        res = np.random.choice(data)
        df = pd.read_csv("./data/mid_merge_f/"+res)
        temp_content = set(list(pd.unique(df['A_inst']))+list(pd.unique(df['B_inst'])))
        cpt += 1
        if cpt == 1000:
            print("too many tries")
            return -1
    return res

In [3]:
from ast import literal_eval
def get_max_duid(df):
    df_max = 0
    for s in df["DU ID"]:
            if type(s) == str:
                s = literal_eval(s)
                if type(s) == list:
                    if s == []:
                        continue
                    else :
                        maxs = max(s)
                        if maxs > df_max:
                            df_max = maxs
    return df_max

In [4]:
test = glob("./data/bot_merge_f/*.csv")

for t in test:
    df = pd.read_csv(t)
    for s in df["Grounded CGUs"]: #Opened CGUsGrounded CGUs
        if type(s) == str : 
            row = literal_eval(s)
            for value in row:
                if type(value) != int :
                    print(t)

In [5]:
def add_offset(serie, offset):
    print("OFFSET :", offset)
    print(serie)
    res = []
    for i,row in enumerate(serie):
        if type(row) == str:
            row = literal_eval(row)
            if row != []:
                res += [[v+(100*offset) for v in row]]
            else :
                res+=[]
        else: 
            res += [np.nan]
    print(pd.Series(res))
    return pd.Series(res)

In [6]:
transitions = [
    "Okay I am moving now.",
    "Alright, I'm relocating now.",
    "I'm moving to the next room now.",
    "let me change the room.",
    "I'm walking to a different area now.",
    "Give me a moment, I'm moving to another spot.",
    "I'm heading to a new location.",
    "I'm shifting to a different room.",
    "I'm stepping into another room.",
]

transitions_directed = ["I'm heading to the south place now.",
    "moving to south now.",
    "Just a sec, I'm moving to the south now.",
    "I'm heading to the east place now.",
    "moving to east now.",
    "Just a sec, I'm moving to the east now.",
    "I'm heading to the north place now.",
    "moving to north now.",
    "Just a sec, I'm moving to the north now.",
    "I'm heading to the west place now.",
    "moving to west now.",
    "Just a sec, I'm moving to the west now."
]

In [7]:

def add_transition():
    if np.random.random() > 0.5:
        return np.random.choice(transitions_directed)
    else :
        return np.random.choice(transitions)

In [8]:
def combine_meetup(n, t):
    #n number of mids
    #t number of dialogs
    #meetup
    meetup = glob("./data/top_merge/*.csv")
    meetup = [m.split("/")[-1] for m in meetup]
    shuffle(meetup)
    
    dataset = []
    #meetup 400 
    #spot   200 
    for i in range(t):
        names = []
        dfs = []
        content = set()
        top = i % len(meetup)
        names += [meetup[top].split("_")[-1].split(".")[0]]
        top_df = pd.read_csv("./data/top_merge/"+meetup[top])
        dfs += [top_df]
        content = content.union(list(pd.unique(top_df['A_inst']))+list(pd.unique(top_df['B_inst'])))
        temp_meetup = np.setdiff1d(np.array(meetup), meetup[top])
        mid = []
        
        #print(du_id_offset)
        # need to check at each step the possible data to chose (same photo not appearing 2 times in dialogue)
        for j in range(n-1):
            #chose a bot dialogue 
            if j == n-2:
                next_dial = pick_next_dialogue(temp_meetup, content)
                #print(next_dial)
                temp_meetup = np.setdiff1d(np.array(temp_meetup), next_dial)
                bot_df = pd.read_csv("./data/bot_merge_f/"+next_dial)
                names += [next_dial.split("_")[-1].split(".")[0]]
                # DU IDs
                bot_df.loc[:, "DU ID"]  = add_offset(bot_df["DU ID"], j+1)
                bot_df.loc[:, "Grounded CGUs"]  = add_offset(bot_df["Grounded CGUs"], j+1)
                bot_df.loc[:, "Opened CGUs"]  = add_offset(bot_df["Opened CGUs"], j+1)
                
                content = content.union(list(pd.unique(bot_df['A_inst']))+list(pd.unique(bot_df['B_inst'])))
                mid += [next_dial]

                #duplication for transition
                dfs[-1] = pd.concat([dfs[-1], dfs[-1].iloc[-1:]], ignore_index=True)
                dfs[-1].loc[dfs[-1].index[-1], "msg"] = add_transition() #TODO maybe more intelligent than duplicate ?
                
                dfs += [bot_df]


            #chose mid dialogues
            else :
                next_dial = pick_next_dialogue(temp_meetup, content)
                names += [next_dial.split("_")[-1].split(".")[0]]
                #print(next_dial)
                temp_meetup = np.setdiff1d(np.array(temp_meetup), next_dial)
                mid_df = pd.read_csv("./data/mid_merge_f/"+next_dial)

                
                mid_df.loc[:, "DU ID"]  = add_offset(mid_df["DU ID"], j+1)
                mid_df.loc[:, "Grounded CGUs"]  = add_offset(mid_df["Grounded CGUs"], j+1)
                mid_df.loc[:, "Opened CGUs"]  = add_offset(mid_df["Opened CGUs"], j+1)

                content = content.union(list(pd.unique(mid_df['A_inst']))+list(pd.unique(mid_df['B_inst'])))
                mid += [next_dial]
                dfs[-1] = pd.concat([dfs[-1], dfs[-1].iloc[-1:]], ignore_index=True)
                dfs[-1].loc[dfs[-1].index[-1], "msg"] = add_transition()
                dfs += [mid_df]
        print(names)
        dfs = pd.concat(dfs)
        path_save = "./data/merged_meetup/merge_"
        for j, name in enumerate(names) :
            if j == len(names) -1 :
                path_save += name + ".csv"
            else :
                path_save += name + "_"
        dfs.to_csv(path_save, index=False)
        dataset += [dfs]
    return dataset
datatest = combine_meetup(4,400)

OFFSET : 1
0       [1]
1    [1, 2]
2    [2, 3]
3    [3, 4]
4    [4, 5]
5    [5, 6]
Name: DU ID, dtype: object
0         [101]
1    [101, 102]
2    [102, 103]
3    [103, 104]
4    [104, 105]
5    [105, 106]
dtype: object
OFFSET : 1
0     []
1    [1]
2    [2]
3    [3]
4    [4]
5    [5]
Name: Grounded CGUs, dtype: object
0    [101]
1    [102]
2    [103]
3    [104]
4    [105]
dtype: object
OFFSET : 1
0    [1]
1    [2]
2    [3]
3    [4]
4    [5]
5    [6]
Name: Opened CGUs, dtype: object
0    [101]
1    [102]
2    [103]
3    [104]
4    [105]
5    [106]
dtype: object
OFFSET : 2
0                    [1]
1                 [2, 3]
2                 [2, 3]
3                    [4]
4                    [5]
5                    [5]
6                    [6]
7                    [7]
8     [3, 4, 5, 6, 7, 9]
9                   [10]
10           [9, 10, 11]
11              [11, 12]
12                  [13]
13                  [14]
Name: DU ID, dtype: object
0                              [201]
1       

KeyboardInterrupt: 

In [ ]:

datatest[0].to_csv("./data/temp.csv", index=False)

In [ ]:
#[d.to_csv("./data/merged_meetup/merge_"+ str(i) +".csv", index= False) for i,d in enumerate(datatest)]